# Finding The Words - Word Associations, Word Embeddings and the Word2Vec model
##### David Miller - August 2018 - [Link to Github](https://github.com/millerdw/millerdw.github.io/tree/master/_notebooks/FindingTheWords_2)
---

## Introduction
In a previous post on [simple NLP using R](http://millerdw.github.io/_posts/2018-07-23-RSS and Simple Natural Language Processing.html), we developed an algorithm that used frequency analysis of the vocabulary in different texts to cluster those texts together. We looked at a couple of improvements on this theme, including using w-shingling to compare more complex word combinations rather than just vocabulary.

In this post I want to develop a couple of those ideas further, and address a few of the shortcomings of those approaches; namely that they are relatively lightweight, and attempt a rather superficial form of unsupervised learning, rather than diving deeper into the meaning of the texts.

Finally, I'm going to take a little bit of a dive into the Word2Vec model ...

- Word Associations
    + Word Roots and Synonyms
- Word Embeddings
    + Translating words into 'meaning' vectors
- Transfer Learnings
    + Using the Word2Vec model

## Word Associations
An immediate downside to comparing complete words and vocabulary within texts is that your algorithm is at the mercy of the author; idioms, favoured words, spelling *conventions* (think 'colour' vs 'color'), spelling *mistakes*... If an algorithm doesn't know to recognise the similarities between such differences (i.e. hasn't been explicitly coded, or taught, to do so), then these all add up to a lot of noise in the signal you are trying to process. This can be a serious problem in texts that are condensed, and have very few words to go by, such as a news article.

This is an important point to consider when you're working with an NLP problem. Do I try to solve it by preprocessing the data before it reaches my algorithm? Do I try an algorithm that's more robust to some of these issues, say focussing on strings of characters rather than complete words? I think both are viable options, depending on what your goals are, but for the purposes of this post, I'm going to focus on the former. In short, because I'm more interested in document-level text comparison, I think a character-level algorithm is likely to be overkill, and added to this I'm a big believer in a plug-and-play approach to programming, whereby the different components of an algorithm can be separated (see *pre*-processing), upgraded, replaced, generally-messed-with, forgotten, and even reintroduced at a later date, *without affecting any code elsewhere in the project*. 


> I'm a big believer in a plug-and-play approach to programming, whereby the different components of an algorithm can be separated (see *pre*-processing), upgraded, replaced, generally-messed-with, forgotten, and even reintroduced at a later date, *without affecting any code elsewhere in the project*.

### Variety is the Spice of Life
A lot of work has been published around the idea of cleaning or normalising individual words before they're input into an algorithm. This is often referred to as [stemming or lemmatization](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html) the text, and consists of either simply removing suffices until only the core of a word remains (stemming), or performing a more complex analysis over a large vocabulary in order to group the various forms of a word together (lemmatization).


Examples...


For the purposes of demonstration, I'm going to use the [Porter Stemmer](http://stp.lingfil.uu.se/~marie/undervisning/textanalys16/porter.pdf) algorithm to cleanup our vocabulary. The Porter Stemmer works by applying a set of rules or heuristics in order, to accurately reduce as many words as possible to a 'correct' word stem. However, the English language is rather varied - given its [variety of historical influences](https://www.merriam-webster.com/help/faq-history), and England's more recent history of global trade, colonialism, and various forms of cultural osmosis (see ['pukka'](https://blog.oxforddictionaries.com/2013/06/14/pukka/), ['tattoo'](https://www.tattoo.com/blog/origin-word-tattoo/), ['chit'](https://www.etymonline.com/word/chit)) - which means that such a set of rules will never be perfect. 

There are lots of different stemming algorithms, but this one is the most widely known, and has the advantage of being published in its own library; [Snowball](http://snowballstem.org/).


> *"We don't just borrow from other languages; English pursues them down alleyways, beats them unconscious, and rifles their pockets for loose grammar."*

### Getting to work
For the purposes of this blog, Python is the programming language of choice. Most of my previous NLP work has been in R, so this will be a first for me. The main reason for my doing this is to make use of the wide variety of tools in the Python community, especially libraries such as [Numpy](http://www.numpy.org/), [Plotly](https://plot.ly/), the [Natural Language Toolkit (NLTK)](https://www.nltk.org/index.html), and later, perhaps, [Keras](https://keras.io/) for deep learning. 

While the Snowball library is included in NLTK, it is also available in a lighter python wrapper called 'PyStemmer'. As with all of these 3rd party libraries, you'll have to install a copy in addition to your Python installation if you're doing this at home. You'll need to open a `cmd` terminal (in Windows) and run `pip install PyStemmer` to install the library, and then import it into the relevant Python script, as usual.

Believe it or not, this will also be one of my first attempts to build a Python notebook from scratch, so let me know if you find something that's bad practice or poorly executed - I've been working with C#, a very similar language, for my whole career, and I've had plenty of experience in reading, contemplating, and occasionally fixing other people's work in Python, but I've never had the joy of starting from a blank piece of screen! 

Anyway, here goes... In the script below, we're going to import the Snowball/PyStemmer library containing the Porter Stemmer algorithm, and apply it to a set of similar-sounding words. Note also that we have the choose the language that we're interested in. This might seem obvious, but it highlights something worth remembering; the algorithm uses a set of fixed rules based on the cases, tenses and plural forms of various words, and these will obviously change depending on the language or dialect used.

In [1]:
import numpy as np
import Stemmer as ps

# list available stemmers
print('Stemmer Languages available:\t',ps.algorithms())


Stemmer Languages available:	 ['danish', 'dutch', 'english', 'finnish', 'french', 'german', 'hungarian', 'italian', 'norwegian', 'porter', 'portuguese', 'romanian', 'russian', 'spanish', 'swedish', 'turkish']


In [2]:
# create stemmer class
stemmer = ps.Stemmer('english')

# stemmer examples
rawWords = ['general','generalizing','generalization','generalise','generalising','generalisation']

print('Raw Word\t Stemmed Word')
for w in rawWords :
    print(w,'\t',stemmer.stemWord(w))

Raw Word	 Stemmed Word
general 	 general
generalizing 	 general
generalization 	 general
generalise 	 generalis
generalising 	 generalis
generalisation 	 generalis


... so it doesn't always work perfectly, see above where it treats the British and Americanised words as having separate word stems. This is because the stemming algorithm uses human-defined rules rather than, say, learning the relationships for itself. Clearly, there are only so many exceptions that can be accounted for, and the specific rules that would stem the British spellings of `generalise` into the word `general` are either too complicated to include or haven't been added yet.

## [Extra] JSON, NewsAPI and building a news dataset

In [4]:
from newsapi import NewsApiClient

news = NewsApiClient(api_key='2bd0b9a9d4594be6b0ceaa26d1861165')

all_news = []
for i in range(1,11):
    all_news.append(news.get_everything(sources='bbc-news,the-verge,abc-news,ary news,associated press,wired,aftenposten,bbc news,bild,blasting news,bloomberg,business insider,engadget,google news,the verge',
                                        from_param='2018-10-01',
                                        to='2018-10-14',
                                        language='en',
                                        page_size=100,
                                        page=i))

In [5]:
import requests
from bs4 import BeautifulSoup as bs

def scrapeArticleUrl(url):
    page = requests.get(url)
    soup = bs(page.text, 'html.parser')
    return [ p.get_text()+' endofpar ' for p in soup('p') ]

rawArticles=[]
for news in all_news:
    rawArticles=rawArticles+news['articles']

# dict comprehension syntax - similar to usage in f#, or the foreach library in R
rawArticles = {i : rawArticles[i] for i in range(len(rawArticles))}

#
rawContents = scrapeArticleUrl(rawArticles[1]['url'])
#rawContents = {i : scrapeArticleUrl(rawArticles[i]['url']) for i in range(len(rawArticles))}
print(rawContents)
print(len(rawArticles))
print(rawArticles[1])
print(rawContents[1])

[' endofpar ', 'Share this with endofpar ', 'Email endofpar ', 'Facebook endofpar ', 'Messenger endofpar ', 'Messenger endofpar ', 'Twitter endofpar ', 'Pinterest endofpar ', 'WhatsApp endofpar ', 'LinkedIn endofpar ', 'Copy this link endofpar ', 'These are external links and will open in a new window endofpar ', 'Life on the humanitarian frontline is not as you know it, says former BBC journalist Mark Doyle, who now works in the aid sector and gives his personal view on the job done by aid workers in Zambia. endofpar ', 'The men in the photo above may not conform to the classic image of aid workers in Africa. endofpar ', 'A more traditional shots would show a nurse caring for a sick child - and the nurse would quite likely be a visiting European. endofpar ', 'But the vast majority of people who run aid projects on the continent are in fact Africans. endofpar ', 'And a huge amount of their time is necessarily devoted to confronting the difficulties of delivering assistance in remote pl

In [6]:
import re

def preprocessArticles(articles) :
    reEndOfSentence = re.compile('\\. ')
    reNonAlphaNumeric = re.compile('\'s|/\n/|/\t/|[\W]+ | ')
    processedArticles = {}
    for i,article in articles.items() :
        # collect contents
        contents = [str(article['description'])+' ']+scrapeArticleUrl(article['url'])
        # convert contents into words
        words = []
        for text in contents:
            # covert to lower case
            text = text.lower()
            # replace full stops with ENDOFSEN
            text = reEndOfSentence.sub(' endofsen ',text)
            # split text into individual words
            newWords = text.split(' ')
            # remove remaining non-alphanumerics
            newWords = [reNonAlphaNumeric.sub('',word) for word in newWords]
            words = words + [word for word in newWords if word!='']

        # combine into list of processed articles
        processedArticles[i] = words
        
    return processedArticles

def stemText(words) :
    return [stemmer.stemWord(word) for word in words]

def stemArticles(articles) :
    return {i:stemText(words) for i,words in articles.items()}


articles = preprocessArticles(rawArticles)
stemmedArticles = stemArticles(articles)

print('\nRaw Description:')
print(rawArticles[1]['description'])
print('\nPreprocessed Description:')
print(articles[1])
print('\nStemmed Description:')
print(stemmedArticles[1])



Raw Description:
From fighting fires to digging cars out of flooded roads - the unglamorous realities of the life of Africa's aid workers.

Preprocessed Description:
['from', 'fighting', 'fires', 'to', 'digging', 'cars', 'out', 'of', 'flooded', 'roads', '-', 'the', 'unglamorous', 'realities', 'of', 'the', 'life', 'of', 'africa', 'aid', 'workers', 'endofsen', 'endofpar', 'share', 'this', 'with', 'endofpar', 'email', 'endofpar', 'facebook', 'endofpar', 'messenger', 'endofpar', 'messenger', 'endofpar', 'twitter', 'endofpar', 'pinterest', 'endofpar', 'whatsapp', 'endofpar', 'linkedin', 'endofpar', 'copy', 'this', 'link', 'endofpar', 'these', 'are', 'external', 'links', 'and', 'will', 'open', 'in', 'a', 'new', 'window', 'endofpar', 'life', 'on', 'the', 'humanitarian', 'frontline', 'is', 'not', 'as', 'you', 'know', 'it,', 'says', 'former', 'bbc', 'journalist', 'mark', 'doyle,', 'who', 'now', 'works', 'in', 'the', 'aid', 'sector', 'and', 'gives', 'his', 'personal', 'view', 'on', 'the', 'job'

## Vectorised Representations

In my previous post, we built an algorithm that looked at the differences between articles in order to cluster them together. In that example, we only needed to generate a difference score between the vocabulary used in two texts, and 

We didn't have to describe the articles outside of this comparison. This simplified matters for us, but did limit the further tools available to us, 

very simple, vector defines whole text

Because stemming makes similar words match with each other, this should reduce the size of the vocabulary.



In [7]:
def buildVocabulary(processedArticles):
    # generate list of all vocabulary
    vocabulary = []
    for i,article in processedArticles.items():
        vocabulary = vocabulary + article
    
    vocabulary = list(set(vocabulary))
    vocabulary.sort(key=str)

    #return as dictionary
    return {i:vocabulary[i] for i in range(len(vocabulary))}

def vectoriseText(vocabToIndexMap,article):
    # encode words as their index in the vocabulary (e.g. possibly 'aardvark' => 23)
    indexArray=[vocabToIndexMap[word] for word in article]
    # transform each
    frequencyVector=[float(np.sum([i==j for j in indexArray])) for i in range(len(vocabToIndexMap))]
    return frequencyVector/np.linalg.norm(frequencyVector)

def vectoriseArticles(processedArticles):
    # build vocabulary
    vocabulary = buildVocabulary(processedArticles)
    # create map of word to vocabulary index
    vocabToIndexMap={w:i for i,w in vocabulary.items()}
    # convert to 2d numpy array of vectors
    vectorisedArticles = np.vstack([vectoriseText(vocabToIndexMap,article) for i,article in processedArticles.items()])
    return vectorisedArticles, vocabulary
    
  
vectorisedArticles, vocabulary = vectoriseArticles(articles)
vectorisedStemmedArticles, stemmedVocabulary = vectoriseArticles(stemmedArticles)

print('Original Vocabulary:\t'+str(vectorisedArticles.shape))
print('Stemmed Vocabulary:\t'+str(vectorisedStemmedArticles.shape))

Original Vocabulary:	(1000, 37270)
Stemmed Vocabulary:	(1000, 30752)


The numbers above show the size of our matrices. Dimension 1, the number or rows, should equal roughly 1000 in both cases; while dimension 2, the number of columns, will vary depending on the number of the size of the vocabulary used in each case.

**As expected, the original vocabulary (untreated) is significantly higher than the stemmed vocabulary.**

Now, each of these sets of `vectorisedArticles` is a table, or matrix, with one row for every article, and one column for every word in the vocabulary, the value in each cell, or element, is the number of times the word (column) appears in the article (row).

This gives us an n-dimensional map of where ....

In [8]:
import time

def singleCalculationTime(vectorisedArticles):
    timeStart=time.process_time()
    totalDistances=np.dot(vectorisedArticles,vectorisedArticles.T)
    timeEnd=time.process_time()
    return (timeEnd-timeStart)

print('Calculation Time, Original Vectors: '+str(singleCalculationTime(vectorisedArticles)))

print('Calculation Time, Stemmed Vectors: '+str(singleCalculationTime(vectorisedStemmedArticles)))


Calculation Time, Original Vectors: 2.5
Calculation Time, Stemmed Vectors: 2.0625


In [ ]:
def winsoriseWordVectors(vectorisedArticles,vocabulary):
    wordFrequency = np.sum(vectorisedArticles,axis=0)
    #print(wordFrequency.shape)
    #print(np.max(wordFrequency))
    #print(np.median(wordFrequency))
    wordFrequencyDeciles = np.percentile(wordFrequency,[i for i in range(100)], interpolation='higher')
    #print(wordFrequencyDeciles)
    columnIndeces=[i for i in range(wordFrequency.shape[0]) if wordFrequency[i]>wordFrequencyDeciles[20] and wordFrequency[i]<=wordFrequencyDeciles[80]]
    #print(columnIndeces)
    return vectorisedArticles[:,columnIndeces],{i:vocabulary[columnIndeces[i]] for i in range(len(columnIndeces))}

winsrVectorArticles,winsrVocabulary = winsoriseWordVectors(vectorisedArticles,vocabulary)
winsrVectorStemmedArticles,winsrStemmedVocabulary = winsoriseWordVectors(vectorisedStemmedArticles,stemmedVocabulary)

print('Winsorised Original Vocabulary:\t'+str(winsrVectorArticles.shape))
print('Calculation Time, Winsorised Original Vectors: '+str(singleCalculationTime(winsrVectorArticles)))
print('Winsorised Stemmed Vocabulary:\t'+str(winsrVectorStemmedArticles.shape))
print('Calculation Time, Winsorised Stemmed Vectors: '+str(singleCalculationTime(winsrVectorStemmedArticles)))

## Vectorised k-Means


cosine similarity, automatically normalises

In [ ]:
def cosineSimilarity(vectorA,vectorB):
    vectorA = [float(element) for element in vectorA]
    vectorB = [float(element) for element in vectorB]
    return np.dot(vectorA,vectorB)/(np.linalg.norm(vectorA)*np.linalg.norm(vectorB))

def centroidDistances(centroids,article):
    return { k:cosineSimilarity(centroid,article) for k,centroid in centroids.items() }

def generateCentroid(vectorisedArticles):
    randId=np.random.randint(vectorisedArticles.shape[0], size=1)
    return vectorisedArticles[randId]

def kMeansCluster(vectorisedArticles,K,G):

    centroids = np.vstack([ generateCentroid(vectorisedArticles) for k in range(K) ])
    articleCentroidIds = []
    performance=[]
    g = 0
    centroidsChanged=True
    while (g<G and centroidsChanged):
        # generate simple cosine distance on normalised vectors between all articles and centroids
        centroidDistances=1-np.dot(vectorisedArticles,centroids.T)
        #if g==0: print(centroidDistances.shape)
        #if g==0: print(centroidDistances)

        # use numpy argmin to find index (column) of nearest centroid by article (row)
        articleCentroidIds=np.argmin(centroidDistances,axis=1)
        articleCentroidDistances=np.min(centroidDistances,axis=1)
        #if g==0: print(articleCentroidIds.shape)
        #if g==0: print(articleCentroidDistances)
        #if g==0: print(articleCentroidIds)
        
        # create new centroids by averaging positions of all article vectors in cluster
        newCentroids=[]
        for k in range(K):
            clusterMembers=vectorisedArticles[[i for i in range(articleCentroidIds.shape[0]) if articleCentroidIds[i]==k],:]
            if g==G-1: print(str(k)+'\t'+str(clusterMembers.shape))
            clusterSize = clusterMembers.shape[0]
            if clusterSize==0:
                newCentroid=generateCentroid(vectorisedArticles)
            else:
                newCentroid=np.mean(clusterMembers,axis=0)
            newCentroids.append(newCentroid)
        newCentroids=np.vstack(newCentroids)

        # update existing centroids
        #print(centroids-newCentroids)
        
        centroidsChanged = (np.sum(np.diag(np.dot(centroids,newCentroids.T))<1.0)>0)
        centroids=newCentroids
        g+=1
        
        interCentroidDistance = np.sum(np.sum(1-np.dot(newCentroids,newCentroids.T)))/(2*K*(K-1))
        intraCentroidDistance = np.mean(articleCentroidDistances)
        performance.append([g,interCentroidDistance,intraCentroidDistance,intraCentroidDistance/interCentroidDistance])
        
    print('Last Generation:\t'+str(g))
    return articleCentroidIds,centroids,np.vstack(performance)



In [ ]:
K=50
G=10000
articleCentroidIds,centroids,performance = kMeansCluster(winsrVectorArticles,K,G)

In [ ]:
performance[1:10,]

In [ ]:
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

plotly.offline.iplot({
    "data": [go.Scatter(x=performance[:,0], y=performance[:,3])],
    "layout": go.Layout(title="hello world")
})

In [163]:
k=4
for i in range(articleCentroidIds.shape[0]):
    if articleCentroidIds[i]==k:
        print(articles[i][1:20])

['officially', 'named', '18-year-old', 'taylor', 'smith', 'as', 'a', 'suspect', 'in', 'the', 'frightening', 'video', 'where', 'jordan', 'holgerson', 'is', 'pushed', 'off', 'a']
['jordanian', 'immigrant', 'has', 'been', 'sentenced', 'to', 'death', 'in', 'texas', 'for', 'the', 'fatal', 'shootings', 'of', 'his', 'son-in-law', 'and', 'daughter', 'best']
['croatia', 'striker', 'mario', 'mandzukic', 'announces', 'his', 'retirement', 'from', 'international', 'football', 'at', 'the', 'age', 'of', '32', 'endofsen', 'endofpar', 'endofpar', 'endofpar']
['fla', 'endofsen', '--', 'the', 'miami', 'dolphins', 'could', 'be', 'without', 'devante', 'parker', 'for', 'some', 'time', 'after', 'the', 'receiver', 'suffered', 'a']
['world', 'heavyweight', 'champion', 'tyson', 'fury', 'says', 'his', 'old', 'self', 'is', '"never', 'to', 'be', 'seen', 'again', 'in', 'history"', 'as', 'he']
['exclusion', 'of', 'an', 'autistic', 'boy', 'after', 'he', 'hit', 'a', 'teaching', 'assistant', 'with', 'a', 'ruler,', 'pun

The trouble is, this doesn't work very well...

Could be to do with the size of the vectors...

plot cluster quality with generations

## Word Embeddings
Superficial text vs deeper meaning

Text frequency vectors imply all words are distinct and orthogonal, which is not true. Tunguska is strongly related to the words Incident and Meteor, despite being superficially dissimilar to both

This is due to **context** and **understanding** of the words